# **AUTOMATSKO RASUDJIVANJE**

- 

In [1]:
import copy

In [91]:
class Formula:
  def __init__(self):
      self.components = []

  def interpret(self, valuation):
    pass

  def copy(self):
    return copy.deepcopy(self)

  def __repr__(self):
    return str(self)

  def __eq__(self, right):
    return Eq(self.copy(), right.copy())

  def __and__(self, right):
    return And(self.copy(), right.copy())

  def __or__(self, right):
    return Or(self.copy(), right.copy())

  def __rshift__(self, right):
    return Impl(self.copy(), right.copy())

  def __invert__(self):
    return Not(self.copy())

  def get_all_variables(self):
    result = set()
    for c in self.components:
      result.update(c.get_all_variables())

    return result

  def is_valid(self):
    variables = list(self.get_all_variables())
    for valuation in all_valuations(variables):
      if self.interpret(valuation) == False:
        return False, valuation 

    return True, None

  def is_satisable(self):
    variables = list(self.get_all_variables())
    for valuation in all_valuations(variables):
      if self.interpret(valuation) == True:
        return True, valuation

    return False, None 

  # ???
  def is_contradictory(self):
    variables = list(self.get_all_variables())
    for valuation in all_valuations(variables):
      if self.interpret(valuation) == True:
        return False, valuation

    return True, None

  def is_falsifiable(self):
    variables = list(self.get_all_variables())
    for valuation in all_valuations(variables):
      if self.interpret(valuation) == False:
        return True, valuation

    return False, None    

  def all_satisfable_valuations(self):
    result = []
    variables = list(self.get_all_variables())
    for valuation in all_valuations(variables):
      if self.interpret(valuation) == True:
        result.append(valuation)

    return result  

class Var(Formula):
  def __init__(self, name):
    super().__init__()
    self.name = name
 
  def interpret(self, valuation):
    return valuation[self.name]

  def get_all_variables(self):
    return set([self.name])

  def __str__(self):
    return self.name

class Const(Formula):
  def __init__(self, value):
    super().__init__()
    self.value = value

  def interpret(self, valuation):
    return self.value

  def __str__(self):
    return f'{1 if self.value else 0}'

class Binary(Formula):
  def __init__(self, left, right):
    super().__init__()
    self.left = left
    self.right = right 

  def interpret(self, valuation):
    return valuation[self.name]


class And(Formula):
  def __init__(self, left, right):
    super().__init__()
    self.components = [left, right]

  def interpret(self, valuation):
    return self.components[0].interpret(valuation) and self.components[1].interpret(valuation)

  def __str__(self):
    return f'({self.components[0]}) & ({self.components[1]})'
  
  """
  def get_all_variables(self):
    left_vars = self.components[0].get_all_variables()
    right_vars = self.components[1].get_all_variables()

    return left_vars.union(right_vars)
  """

class Or(Formula):
  def __init__(self, left, right):
    super().__init__()
    self.components = [left, right]

  def interpret(self, valuation):
    return self.components[0].interpret(valuation) or self.components[1].interpret(valuation)

  def __str__(self):
    return f'({self.components[0]}) | ({self.components[1]})'

  """
  def get_all_variables(self):
    left_vars = self.components[0].get_all_variables()
    right_vars = self.components[1].get_all_variables()

    return left_vars.union(right_vars)
  """

class Impl(Formula):
  def __init__(self, left, right):
    super().__init__()
    self.components = [left, right]

  def interpret(self, valuation):
    return not self.components[0].interpret(valuation) or self.components[1].interpret(valuation)

  def __str__(self):
    return f'({self.components[0]}) >> ({self.components[1]})'

  """
  def get_all_variables(self):
    left_vars = self.components[0].get_all_variables()
    right_vars = self.components[1].get_all_variables()

    return left_vars.union(right_vars)
  """

class Eq(Formula):
  def __init__(self, left, right):
    super().__init__()
    self.components = [left, right]

  def interpret(self, valuation):
    return self.components[0].interpret(valuation) == self.components[1].interpret(valuation) 
  
  def __str__(self):
    return f'({self.components[0]}) == ({self.components[1]})'

  """
  def get_all_variables(self):
    left_vars = self.components[0].get_all_variables()
    right_vars = self.components[1].get_all_variables()

    return left_vars.union(right_vars)
  """

class Not(Formula):
  def __init__(self, operand):
    super().__init__()
    self.components = [operand]

  def interpret(self, valuation):
    return not self.components[0].interpret(valuation)

  def __str__(self):
    return f'~({self.components[0]})'

  """
  def get_all_components(self):
    return self.components[0].get_all_variables()
  """

In [97]:
from itertools import combinations
def all_valuations(variables):
  for r in range(len(variables) + 1):
    for true_variables in combinations(variables, r):
      res = {x : False for x in variables}
      res.update({x : True for x in true_variables})
      yield res 
      # izvrsava je tako sto, kada se prvi put pozove
      # napravi se objekat koji ima spec metodu kojom moze da 
      # vrati sledeci element. Svaki put kada se poziva izvrsava
      # se tamo gde je stao

In [43]:
def vars(names):
  return [Var(name.strip()) for name in names.split(',')]

In [94]:
if __name__ == '__main__':
  x, y, z = vars("x,y,z")

  formula = (x & y) | (z >> y)
  
  # formula = ~x & x 

  # formula = Or(
  #     And(x, y),
  #     Impl(x, y)
  # )

  valuation = {
      "x": True,
      "y": False,
      "z": True
  }

  print(formula)
  print(formula.interpret(valuation))

  print("valid:",formula.is_valid())
  print("satisfable:", formula.is_satisable())
  print("contadictory:", formula.is_contradictory())
  print("falsifiable:", formula.is_falsifiable())
  print(formula.all_satisfable_valuations())

((x) & (y)) | ((z) >> (y))
False
valid: (False, {'z': True, 'y': False, 'x': False})
satisfable: (True, {'z': False, 'y': False, 'x': False})
contadictory: (False, {'z': False, 'y': False, 'x': False})
falsifiable: (True, {'z': True, 'y': False, 'x': False})
[{'z': False, 'y': False, 'x': False}, {'z': False, 'y': True, 'x': False}, {'z': False, 'y': False, 'x': True}, {'z': True, 'y': True, 'x': False}, {'z': False, 'y': True, 'x': True}, {'z': True, 'y': True, 'x': True}]


In [99]:
def evaluate_formula(formula: Formula):
  print(formula)
  # print(formula.interpret(valuation))
  print("valid:",formula.is_valid())
  print("satisfable:", formula.is_satisable())
  print("contadictory:", formula.is_contradictory())
  print("falsifiable:", formula.is_falsifiable())
  print("all true valuations: ")
  for val in all_valuations(formula.get_all_variables()):
      if formula.interpret(val):
          print(val)

In [100]:
"""
  U igri mines dim 2x3 dobijena je sledeca konfiguracija
  |1|A|C|
  |1|B|2|
  A, B, C su neotvorena polja, a brojevi oznacavaju broj mina u okolini polja.
  Zapisati u iskaznoj logici uslove koji moraju da vaze.
"""

A, B, C = vars("A,B,C")
formula = (A | B) & ~(A & B) & (B | A) & ~(B & A) & ~(~A & ~B & ~C) & (A | B) & (B | C) & (A | C) & ~(A & B & C) 

evaluate_formula(formula)

(((((((((A) | (B)) & (~((A) & (B)))) & ((B) | (A))) & (~((B) & (A)))) & (~(((~(A)) & (~(B))) & (~(C))))) & ((A) | (B))) & ((B) | (C))) & ((A) | (C))) & (~(((A) & (B)) & (C)))
valid: (False, {'B': False, 'C': False, 'A': False})
satisfable: (True, {'B': True, 'C': True, 'A': False})
contadictory: (False, {'B': True, 'C': True, 'A': False})
falsifiable: (True, {'B': False, 'C': False, 'A': False})
all true valuations: 
{'B': True, 'C': True, 'A': False}
{'B': False, 'C': True, 'A': True}


In [101]:
"""
|A|B|
|C|D|

Zapisati uslov da se u tabelu 2x2 sa polja A, B, C, D moze postaviti tacno jedan zeton
u svakom redu
"""

A, B, C, D = vars("A,B,C,D")

formula = (A | B) & ~(A & B) & (C | D) & ~(C & D)

evaluate_formula(formula)

((((A) | (B)) & (~((A) & (B)))) & ((C) | (D))) & (~((C) & (D)))
valid: (False, {'B': False, 'D': False, 'C': False, 'A': False})
satisfable: (True, {'B': True, 'D': True, 'C': False, 'A': False})
contadictory: (False, {'B': True, 'D': True, 'C': False, 'A': False})
falsifiable: (True, {'B': False, 'D': False, 'C': False, 'A': False})
all true valuations: 
{'B': True, 'D': True, 'C': False, 'A': False}
{'B': True, 'D': False, 'C': True, 'A': False}
{'B': False, 'D': True, 'C': False, 'A': True}
{'B': False, 'D': False, 'C': True, 'A': True}


In [102]:
"""
U iskaznoj loici zapisati uslov da bitovi 3-bitnog polja moraju biti
jednaki
"""

A, B, C = vars("A,B,C")

formula = (A == B) & (B == C) 

evaluate_formula(formula)

((A) == (B)) & ((B) == (C))
valid: (False, {'B': True, 'C': False, 'A': False})
satisfable: (True, {'B': False, 'C': False, 'A': False})
contadictory: (False, {'B': False, 'C': False, 'A': False})
falsifiable: (True, {'B': True, 'C': False, 'A': False})
all true valuations: 
{'B': False, 'C': False, 'A': False}
{'B': True, 'C': True, 'A': True}


In [103]:
'''
Dva dvobitna broja se sabiraju i daju rezultat 3.
1+2
2+1
3+0
0+3
    A B
    C D
    ---
    1 1
'''

A,B,C,D = vars("A,B,C,D")

formula = (B | D) & ~(B & D) & (A | C) & ~(A & C) 

evaluate_formula(formula)

((((B) | (D)) & (~((B) & (D)))) & ((A) | (C))) & (~((A) & (C)))
valid: (False, {'B': False, 'D': False, 'C': False, 'A': False})
satisfable: (True, {'B': True, 'D': False, 'C': True, 'A': False})
contadictory: (False, {'B': True, 'D': False, 'C': True, 'A': False})
falsifiable: (True, {'B': False, 'D': False, 'C': False, 'A': False})
all true valuations: 
{'B': True, 'D': False, 'C': True, 'A': False}
{'B': True, 'D': False, 'C': False, 'A': True}
{'B': False, 'D': True, 'C': True, 'A': False}
{'B': False, 'D': True, 'C': False, 'A': True}


In [104]:
'''
U iskoznoj logici zapisati da je 4 bitna reprezentacija broja palindrom ali da 
bitovi nisu jednaki
ABCD
'''

A,B,C,D = vars("A,B,C,D")

formula = (A == D) & (B == C) & (~(A == B) & (B == C) & (C == D))

evaluate_formula(formula)

(((A) == (D)) & ((B) == (C))) & (((~((A) == (B))) & ((B) == (C))) & ((C) == (D)))
valid: (False, {'C': False, 'B': False, 'D': False, 'A': False})
satisfable: (False, None)
contadictory: (True, None)
falsifiable: (True, {'C': False, 'B': False, 'D': False, 'A': False})
all true valuations: 


In [105]:
'''
Tri polja se boje crvenom ili plavom. 
Ukoliko je prvo crveno, druga dva moraju biti iste boje.
Ukoliko je drugo crveno, trece mora biti plavo.
'''

A, B, C = vars("A,B,C")

formula = (A >> (B == C)) & (B >> ~C)
evaluate_formula(formula )

((A) >> ((B) == (C))) & ((B) >> (~(C)))
valid: (False, {'B': True, 'C': True, 'A': False})
satisfable: (True, {'B': False, 'C': False, 'A': False})
contadictory: (False, {'B': False, 'C': False, 'A': False})
falsifiable: (True, {'B': True, 'C': True, 'A': False})
all true valuations: 
{'B': False, 'C': False, 'A': False}
{'B': True, 'C': False, 'A': False}
{'B': False, 'C': True, 'A': False}
{'B': False, 'C': False, 'A': True}
